**POC for identifying groups of individuals who can live together at a new dwelling**


In [159]:
# imports
import os
import pandas as pd
import numpy as np

In [160]:
# load in CSV of Persons
# expect single CSV to have all necessary info about a person
people_filepath = 'data/PeopleSheet.csv'
peeps = pd.read_csv(people_filepath)

In [161]:
# Create a Dwelling to test People against
# Note: no geo math, just flag whether, e.g., it is no_offenders (True)
dwelling_filepath = 'data/DwellingSheet.csv'
dwelling = pd.read_csv(dwelling_filepath)

In [162]:
# Test Persons against Dwelling, return list of eligible and eliminated
if not dwelling.offender_ok[0] == 'y':
    eligibles_loc = peeps[peeps['is_offender'] == 'n']  ## fragile: uppercase breaks
    eliminated_loc = peeps[peeps['is_offender'] == 'y']

print(f'# eligible: {len(eligibles_loc)}')
print(f'# eliminated: {len(eliminated_loc)}')

if len(eligibles_loc) + len(eliminated_loc) == len(peeps):
    print(f'All {len(peeps)} peeps accounted for!')    
else:
    print('Looks like we might have lost one! Started with X peeps.')
# test that all records went one place or the other (using .len?)

# eligible: 6
# eliminated: 1
All 7 peeps accounted for!


**HARD GROUPING**

Hard-group eligible persons based on a small number of lifestyle flags

**Hard grouping within DF**

In [163]:
# add 'group' column to peeps, initialize to 0
eligibles_loc.insert(1, 'hard_group', -1)  # can only do this once per run or errors out

In [164]:
for index, peep in eligibles_loc.iterrows():
    check_group = 0 # for each peep, check fit starting with group 0
    # any groups left to test fit with? If not, must need another group for this peep
    if check_group > eligibles_loc.hard_group.max():
        print(f'new group please')
        eligibles_loc.hard_group[eligibles_loc.UID == peep.UID] = check_group
    # otherwise, see if peep is compatible with members of current group, by iterating through group
    else:
        for index, groupie in eligibles_loc[eligibles_loc.hard_group == check_group].iterrows():
            if groupie.is_smoker == 'y' and peep.smoker_ok == 'n':
                check_group = check_group + 1
                break
        # if it didn't 'break', the group fit is okay; so assign peep to this group
        eligibles_loc.hard_group[eligibles_loc.UID == peep.UID] = check_group
             
            



#     for checkgroup in range(len(peeps[
    # iterate groups
    
    # iterate peeps in group, check flags for new person
# for index, peep in eligibles_loc.iterrows():
#     print(peep.UID, peep.hard_group)


new group please
/Users/wes/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/wes/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [165]:
eligibles_loc

,UID,hard_group,is_offender,is_smoker,smoker_ok,has_dog,dog_allergy
1,X0002,0,n,y,y,y,n
2,X0003,0,n,y,y,y,n
3,X0004,0,n,y,y,y,n
4,X0005,1,n,y,n,y,n
5,X0006,0,n,y,y,y,n
6,X0007,1,n,n,n,y,y


In [166]:
# Report groups